# Advanced selection rules {#advanced_selection_rules_example}

This example shows how to use advanced rules, including the geometrical,
cut-off, and variable offset rules. It also demonstrates how rules can
be templated and reused with different parameters. For more basic rules,
see `basic_selection_rules_example`{.interpreted-text role="ref"}.

This example only shows the PyACP part of the setup. For a complete
composite analysis, see `pymapdl_workflow_example`{.interpreted-text
role="ref"}.


# Import modules

Import the standard library and third-party dependencies.


In [ ]:
import pathlib
import tempfile

import numpy as np
import pyvista

Import the PyACP dependencies.


In [ ]:
from ansys.acp.core import (
    BooleanOperationType,
    EdgeSetType,
    LinkedSelectionRule,
    PhysicalDimension,
    launch_acp,
)
from ansys.acp.core.extras import ExampleKeys, get_example_file

# Start ACP and load the model


Get the example file from the server.


In [ ]:
tempdir = tempfile.TemporaryDirectory()
WORKING_DIR = pathlib.Path(tempdir.name)
input_file = get_example_file(ExampleKeys.MINIMAL_FLAT_PLATE, WORKING_DIR)

Launch the PyACP server and connect to it.


In [ ]:
acp = launch_acp()

Load the model from the input file.


In [ ]:
model = acp.import_model(input_file)
print(model.unit_system)

Add more layers to the modeling ply so that it is easier to see the
effects of the selection rules. Plot the thickness of all the plies
without any rules.


In [ ]:
modeling_ply = model.modeling_groups["modeling_group"].modeling_plies["ply"]
modeling_ply.number_of_layers = 10

model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)

# Parametrized Parallel Rule


Rules can be parametrized. This is useful when a rule is used multiple
times but with different parameters. The
`.LinkedSelectionRule`{.interpreted-text role="class"} class shows what
parameters are available for each rule. This code modifies the extent of
the parallel rule.


Create a parallel rule.


In [ ]:
parallel_rule = model.create_parallel_selection_rule(
    name="parallel_rule",
    origin=(0, 0, 0),
    direction=(1, 0, 0),
    lower_limit=0.005,
    upper_limit=1,
)

Assign it the modeling ply.


In [ ]:
linked_parallel_rule = LinkedSelectionRule(parallel_rule)
modeling_ply.selection_rules = [linked_parallel_rule]

Plot the thickness of the ply before the parametrization.


In [ ]:
model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)

Modify the parallel rule by changing the parameters of the linked rule.
Parameters defined on the linked rule override the parameters of the
original rule.


In [ ]:
linked_parallel_rule.template_rule = True
linked_parallel_rule.parameter_1 = 0.002
linked_parallel_rule.parameter_2 = 0.1

Plot the thickness of the ply with the modified rule.


In [ ]:
model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)

# Create a geometrical selection rule


Add a CAD geometry to the model.


In [ ]:
triangle_path = get_example_file(ExampleKeys.RULE_GEOMETRY_TRIANGLE, WORKING_DIR)
triangle = model.create_cad_geometry()
triangle.refresh(triangle_path)

# Note: It is important to update the model here, because the root_shapes of the
# cad_geometry are not available until the model is updated.
model.update()

Create a virtual geometry from the CAD geometry.


In [ ]:
triangle_virtual_geometry = model.create_virtual_geometry(
    name="triangle_virtual_geometry", cad_components=triangle.root_shapes.values()
)

Create the geometrical selection rule.


In [ ]:
geometrical_selection_rule = model.create_geometrical_selection_rule(
    name="geometrical_rule",
    geometry=triangle_virtual_geometry,
)

Assign the geometrical selection rule to the ply. Plot the ply extent
with the outline of the geometry.


In [ ]:
modeling_ply.selection_rules = [LinkedSelectionRule(geometrical_selection_rule)]
model.update()
assert model.elemental_data.thickness is not None

plotter = pyvista.Plotter()
plotter.add_mesh(
    triangle.visualization_mesh.to_pyvista(), style="wireframe", line_width=4, color="white"
)
plotter.add_mesh(model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh), show_edges=True)
plotter.show()

# Create a cut-off selection rule


Add the cut off CAD geometry to the model.


In [ ]:
cut_off_plane_path = get_example_file(ExampleKeys.CUT_OFF_GEOMETRY, WORKING_DIR)
cut_off_plane = model.create_cad_geometry()
cut_off_plane.refresh(cut_off_plane_path)

# Note: It is important to update the model here, because the root_shapes of the
# cad_geometry are not available until the model is updated.
model.update()

Create a virtual geometry from the CAD geometry.


In [ ]:
cut_off_virtual_geometry = model.create_virtual_geometry(
    name="cut_off_virtual_geometry", cad_components=cut_off_plane.root_shapes.values()
)

Create the cut_off selection rule.


In [ ]:
cut_off_selection_rule = model.create_cut_off_selection_rule(
    name="cut_off_rule",
    cut_off_geometry=cut_off_virtual_geometry,
)

Assign the cut_off selection rule to the ply. Plot the ply extent with
the outline of the geometry.


In [ ]:
modeling_ply.selection_rules = [LinkedSelectionRule(cut_off_selection_rule)]

model.update()
assert model.elemental_data.thickness is not None
plotter = pyvista.Plotter()
plotter.add_mesh(cut_off_plane.visualization_mesh.to_pyvista(), color="white")
plotter.add_mesh(model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh))
plotter.camera_position = [(-0.05, 0.01, 0), (0.005, 0.005, 0.005), (0, 1, 0)]

plotter.show()

# Create a variable offset selection rule


Create the lookup table.


In [ ]:
lookup_table = model.create_lookup_table_1d(
    name="lookup_table",
    origin=(0, 0, 0),
    direction=(0, 0, 1),
)

Add the location data. The \"Location\" column of the lookup table is
always created by default.


In [ ]:
lookup_table.columns["Location"].data = np.array([0, 0.005, 0.01])

Create the offset column that defines the offsets from the edge.


In [ ]:
offsets_column = lookup_table.create_column(
    name="offset",
    physical_dimension=PhysicalDimension.LENGTH,
    data=np.array([0.00, 0.004, 0]),
)

Create the edge set from the \"All_Elements\" element set. Because you
assigned 30 degrees to the limit angle, only one edge at x=0 is
selected.


In [ ]:
edge_set = model.create_edge_set(
    name="edge_set",
    edge_set_type=EdgeSetType.BY_REFERENCE,
    limit_angle=30,
    element_set=model.element_sets["All_Elements"],
    origin=(0, 0, 0),
)

Create the variable offset rule and assign it to the ply.


In [ ]:
variable_offset_rule = model.create_variable_offset_selection_rule(
    name="variable_offset_rule", edge_set=edge_set, offsets=offsets_column, distance_along_edge=True
)

modeling_ply.selection_rules = [LinkedSelectionRule(variable_offset_rule)]

Plot the ply extent.


In [ ]:
model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)

# Create a boolean selection rule


Creating a Boolean selection rule and assigning it to a ply has the same
effect as linking the individual rules directly to the ply. Boolean
rules are still useful because they can help organize rules and make
more complex ones.

Create a cylindrical selection rule to combine with the parallel rule.


In [ ]:
cylindrical_rule_boolean = model.create_cylindrical_selection_rule(
    name="cylindrical_rule",
    origin=(0.005, 0, 0.005),
    direction=(0, 1, 0),
    radius=0.002,
)

parallel_rule_boolean = model.create_parallel_selection_rule(
    name="parallel_rule",
    origin=(0, 0, 0),
    direction=(1, 0, 0),
    lower_limit=0.005,
    upper_limit=1,
)

linked_cylindrical_rule_boolean = LinkedSelectionRule(cylindrical_rule_boolean)
linked_parallel_rule_boolean = LinkedSelectionRule(parallel_rule_boolean)

boolean_selection_rule = model.create_boolean_selection_rule(
    name="boolean_rule",
    selection_rules=[linked_parallel_rule_boolean, linked_cylindrical_rule_boolean],
)

modeling_ply.selection_rules = [LinkedSelectionRule(boolean_selection_rule)]

Plot the ply extent.


In [ ]:
model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)

Modify the operation type of the Boolean selection rule so that the two
rules are added.


In [ ]:
linked_parallel_rule_boolean.operation_type = BooleanOperationType.INTERSECT
linked_cylindrical_rule_boolean.operation_type = BooleanOperationType.ADD

Plot the ply extent.


In [ ]:
model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)